In [2]:
import boto3
import json
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split


In [5]:
house_data=datasets.fetch_california_housing()
df=pd.DataFrame(house_data['data'],columns=house_data['feature_names'])
x=df
y=house_data.target
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)


In [42]:
pd.DataFrame(x_train).iloc[[12]].to_json(orient='split')

'{"columns":["alcohol","malic_acid","ash","alcalinity_of_ash","magnesium","total_phenols","flavanoids","nonflavanoid_phenols","proanthocyanins","color_intensity","hue","od280\\/od315_of_diluted_wines","proline"],"index":[154],"data":[[12.58,1.29,2.1,20.0,103.0,1.48,0.58,0.53,1.4,7.6,0.58,1.55,640.0]]}'

In [14]:
import mlflow
from mlflow.models import infer_signature
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
mlflow.set_experiment('House Price Predcition')
mlflow.set_tracking_uri('http://127.0.0.1:5000/')
with mlflow.start_run(run_name='HouseData') as run:
    lr=LinearRegression()
    lr.fit(x_train,y_train)
    y_pred=lr.predict(x_test)
    accuracy=r2_score(y_test,y_pred)
    mae=mean_absolute_error(y_test,y_pred)
    mse=mean_squared_error(y_test,y_pred)
    mlflow.log_metric('r2_score',accuracy)
    mlflow.log_metric('mae',mae)
    mlflow.log_metric('mse',mse)
    mlflow.log_param('copy_X',lr.get_params()['copy_X'])
    mlflow.log_param('fit_intercept',lr.get_params()['fit_intercept'])
    # mlflow.log_param('dataset_name','California Housing')
    signature=infer_signature(model_input=x_train,model_output=y_pred)
    mlflow.sklearn.log_model(lr,'Logistic Regression',signature=signature)
    



2024/10/21 15:13:57 INFO mlflow.tracking._tracking_service.client: 🏃 View run HouseData at: http://127.0.0.1:5000/#/experiments/292572241966678808/runs/9bad7772a21d452ea6d88fff486f896f.
2024/10/21 15:13:57 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/292572241966678808.


In [13]:
lr.get_params()

{'copy_X': True, 'fit_intercept': True, 'n_jobs': None, 'positive': False}